In [200]:
#!pip install transformers
#!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117

In [1]:
import re
import torch
import pandas as pd

from transformers import GPT2Tokenizer, T5ForConditionalGeneration 
from transformers import AutoTokenizer, T5ForConditionalGeneration
from functools import partial

from IPython.display import display, Markdown
device = 'cuda'

from transformers import set_seed
set_seed(42)

In [2]:
torch.cuda.is_available()

True

In [158]:
columns = ['site_id', 'date', 'title', 'path', 'text_type', 'full_text']
texts = pd.read_csv('legal_acts.csv', names=columns)
texts['full_text'] = texts['full_text'].apply(lambda x: re.sub(' +', ' ', x))
#texts['full_text'] = texts['full_text'].apply(lambda x: re.sub(r"\n", '', x))

In [130]:
columns = ['url', 'text']
texts_norm = pd.read_csv('documents (1).csv', names=columns).dropna()
texts_norm['text'] = texts_norm['text'].apply(lambda x: re.sub(' +', ' ', x))
texts_norm['text'] = texts_norm['text'].apply(lambda x: re.sub(r"\n", '', x))

In [163]:
# Имитация текста от ретривала
retrieved_docs_example = ' '.join(
    texts[2:5].apply(
    lambda x: f'Документ c названием {x["site_id"]} '+ x['full_text'],
    axis=1)
)

In [164]:
len(tokenizer.encode(retrieved_docs_example))

1225

In [204]:
system_prompt = """
Ты помощник по документам Банка России, твоя задача ответить на вопрос пользователя. Далее на вход тебе будут приходить вопросы пользователей
(формат: Вопрос пользователя <вопрос пользователя>) и документы (формат: Документ c названием <название документа> <содержание документа>).
Входные документы содержат в себе ответ на вопрос с большой вероятностью.
Просьбы и уточнения:
1. Очень внимательно отвечай на вопрос, ответ на него может быть очень большим, но скорее всего он содержится в данных тебе документах!
2. Ответ на вопрос может состоять из нескольких пунктов, поэтому, если ты нашел один пункт, то посмотри вперед, возможно их несколько!
3. Если ответ на заданный вопрос не содержится в документах, то НЕ ОТВЕЧАЙ. Пиши выражение "Нет такой информации"
Нужно ответить на вопрос пользователя.
Задание: посмотри на документы, и опираясь на них дай свой ответ на вопрос в конце.
\n"""

document_prompt = retrieved_docs_example

In [192]:
system_prompt

'Ты помощник по документам Банка России, твоя задача ответить на вопрос пользователя. Далее на вход тебе будут приходить вопросы пользователей\n(формат: Вопрос пользователя <вопрос пользователя>) и документы (формат: Документ c названием <название документа> <содержание документа>).\nВходные документы содержат в себе ответ на вопрос с большой вероятностью.\nПросьбы и уточнения:\n1. Очень внимательно отвечай на вопрос, ответ на него может быть очень большим, но скорее всего он содержится в данных тебе документах!\n2. Ответ на вопрос может состоять из нескольких пунктов, поэтому, если ты нашел один пункт, то посмотри вперед, возможно их несколько!\n3. Если ответ на заданный вопрос не содержится в документах, то НЕ ОТВКЕ\nНужно ответить на вопрос пользователя.\nЗадание: посмотри на документы, и опираясь на них дай свой ответ на вопрос в конце.\n\n'

In [188]:
questions = ['какой размер штрафа за нарушение кредитной организацией условий Договора об общих условиях проведения депозитных операций?',
             'Что рекомендует Банк России кредитным организациям в целях снижения рисков использования услуг кредитных организаций в противоправных целях?',
             'В чем разница скуфом и тюбиком?']

In [178]:
full_propmt = system_prompt + document_prompt

In [34]:
torch.cuda.empty_cache()

In [206]:
#model_name = 'ai-forever/FRED-T5-1.7B'
#tokenizer = AutoTokenizer.from_pretrained(model_name)
#model = T5ForConditionalGeneration.from_pretrained(model_name).cuda()

model_name = 'hivaze/AAQG-QA-QG-FRED-T5-1.7B'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name).cuda().eval()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [207]:
def generate_text(prompt, tokenizer, model, n=1, temperature=0.8, num_beams=4):
    encoded_input = tokenizer.encode_plus(prompt, return_tensors='pt')
    encoded_input = {k: v.to(model.device) for k, v in encoded_input.items()}

    resulted_tokens = model.generate(**encoded_input,
                                   max_new_tokens=200,
                                   do_sample=True,
                                   num_beams=num_beams,
                                   num_return_sequences=n,
                                   no_repeat_ngram_size=2,
                                   temperature=temperature,
                                   top_p=0.9,
                                   top_k=50)
    
    resulted_texts = tokenizer.batch_decode(resulted_tokens, skip_special_tokens=True)

    return resulted_texts

generate_text = partial(generate_text, tokenizer=tokenizer, model=model)

In [208]:
set_seed(88)
for i in range(len(questions)):
    QA_PROMPT = system_prompt + "Сгенерируй ответ на вопрос по тексту. Текст: '{context}'. Вопрос: '{question}'."
    answer = generate_text(QA_PROMPT.format(context=document_prompt, question=questions[i]))[0]

    display(Markdown(f'Вопрос {i + 1}: {questions[i]} \n'))
    display(Markdown(answer))

Вопрос 1: какой размер штрафа за нарушение кредитной организацией условий Договора об общих условиях проведения депозитных операций? 


Установить размер штрафа за нарушение кредитной организацией условий Договора об общих условиях проведения депозитных операций равным нулю.

Вопрос 2: Что рекомендует Банк России кредитным организациям в целях снижения рисков использования услуг кредитных организаций в противоправных целях? 


Банк России рекомендует кредитным организациям: 
1. Осуществлять мониторинг операций по банковским счетам дропов или операций с использованием предоставленных им электронных средств платежа в целях установления клиентов – физических лиц, осуществляющих переводы денежных средств (электронных денежных средства) в пользу дронов, связанные с покупкой (продажей) цифровых валют. 

Вопрос 3: В чем разница скуфом и тюбиком? 


Скуф – это средство для снятия макияжа, а тюбик – для 
нанесения.

In [215]:
def llm_answer(question, documents, model, sys_prompt=system_prompt):
    documents_retriever = ''
    for i in range(len(documents)):
        documents_retriever += f'Документ c номером {i}: {documents[i]} \n'
    #temp_prompt = f'<LM>{full_propmt + questions[i]}'
    QA_PROMPT = system_prompt + "Сгенерируй ответ на вопрос по тексту. Текст: '{context}'. Вопрос: '{question}'."
    answer = generate_text(QA_PROMPT.format(context=documents_retriever, question=question))[0]
    return answer

In [216]:
answer = llm_answer(questions[0], [document_prompt], model)
answer

'Установить размер штрафа за нарушение кредитной организацией условий Договора об общих условиях проведения депозитных операций равным нулю.'

### Den4ikAI/FRED-T5-LARGE_text_qa

In [196]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
from transformers import GenerationConfig

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
generation_config = GenerationConfig.from_pretrained("Den4ikAI/FRED-T5-LARGE_text_qa")
tokenizer = AutoTokenizer.from_pretrained("Den4ikAI/FRED-T5-LARGE_text_qa")
model = AutoModelForSeq2SeqLM.from_pretrained("Den4ikAI/FRED-T5-LARGE_text_qa").to(device)
model.eval()

def generate(prompt):
    data = tokenizer(f"{prompt}", return_tensors="pt").to(model.device)
    output_ids = model.generate(**data, generation_config=generation_config)[0]
    out = tokenizer.decode(output_ids.tolist())
    return out

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [182]:
temp_prompt = f'<LM>{full_propmt + questions[i]}'
QA_PROMPT = system_prompt + "Сгенерируй ответ на вопрос по тексту. Текст: '{context}'. Вопрос: '{question}'."
QA_PROMPT.format(context=document_prompt, question=questions[i])

"\nТы помощник по документам Банка России, твоя задача ответить на вопрос пользователя. Далее на вход тебе будут приходить вопросы пользователей\n(формат: Вопрос пользователя <вопрос пользователя>) и документы (формат: Документ c названием <название документа> <содержание документа>).\nВходные документы содержат в себе ответ на вопрос с большой вероятностью.\nПросьбы и уточнения:\n1. Очень внимательно отвечай на вопрос, ответ на него может быть очень большим, но скорее всего он содержится в данных тебе документах!\n2. Ответ на вопрос может состоять из нескольких пунктов, поэтому, если ты нашел один пункт, то посмотри вперед, возможно их несколько!\n3. Если ответ на заданный вопрос не содержится в документах, то НЕ ОТВЕЧАЙ НА ЭТОТ ВОПРОС\nНужно ответить на вопрос пользователя.\nЗадание: посмотри на документы, и опираясь на них дай свой ответ на вопрос в конце.\n\nСгенерируй ответ на вопрос по тексту. Текст: 'Документ c названием 4-МР ЦЕНТРАЛЬНЫЙ БАНК РОССИЙСКОЙ ФЕДЕРАЦИИ \n(БАНК РОССИИ)

In [197]:
prompt = '''<SC6>Текст: {}\nВопрос: {}\nОтвет: <extra_id_0>'''.format(full_propmt, questions[0])
print(generate(prompt))

<pad><extra_id_0>
Размер штрафа за нарушение условий Договора об общих условиях проведения депозитных операций составляет 0,5% от суммы депозита. 
При этом, если сумма депозита превышает сумму штрафа, то размер штрафа увеличивается на 0,5%. 
Если сумма депозита меньше суммы штрафа, то штраф уменьшается на 0,1%. 
В случае, если размер штрафа превышает сумму депозита, то штраф увеличивается на 1%. 
Примечание: размер штрафа может быть уменьшен на 0,2% при условии, что сумма депозита не превышает сумму штрафа. 
Отмена приказа Банка России от 28 апреля 2018 года № ОД-1132 
об отмене приказа от 28 марта 2018 года № ИН-01-20/16 об отмене Информационного письма Банка России о порядке проведения Банком России депозитных операций. 
В целях снижения рисков использования услуг кредитных организаций в противоправных целях, Банк России рекомендует кредитным организациям: 
1. Осуществлять мониторинг операций по банковским счетам дропов или операций с использованием предоставленных им электронных сре